<a href="https://colab.research.google.com/github/teang1995/satellite_image_classification/blob/master/simple_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**import Libraries**

In [0]:
import os
import shutil
from PIL import Image
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
from torchvision import transforms as T
import torchvision
import cv2
import sys
from google.colab import drive
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

**check cuda**

In [2]:
torch.cuda.is_available()

True

**drive mount**

In [3]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Dataset Class**

In [0]:
class NWPU_RESISC45(Dataset):
  '''
  1. Satellite image dataset
  2. 45 classes
  '''
  #initiallize my data , download
  def __init__(self, root, phase = 'train',transform = None):
    self.phase = phase
    #요 부분 코드 수정 필요해보임!
    #현재 data_list_file이 txt형식으로 돼있어보인다.
    #나는 excel혹은 csv파일 꼴로 만들고 싶으니 그에 맞게 수정이 필요함.
    #imgs = [os.path.join(root, img[:-1]) for img in imgs]
    imgs = pd.read_csv(root) # csv파일의 한 행
    self.img_list = [os.path.join(imgs["data_path"][i]) for i in range(len(imgs))] #파일 경로로 리스트를 만듦.
    self.label_list = [os.path.join(imgs["label"][i]) for i in range(len(imgs))] #라벨 리스트를 만듦
    #self.imgs = np.random.permutation(imgs) #shuffle = True로 대체될 것 같기도 함
    normalize = T.Normalize(mean=[0.5], std=[0.5])
    self.transform = transform
  #csv파일에서 따오기로 결정
  def __getitem__(self,index):
    sample = self.img_list[index]
    img_path = sample
    data = Image.open(img_path)
    #data = data.convert('L') #binary chaneel
    data = self.transform(data)
    label = self.label_list[index] 
    return data.float(), label

  def __len__(self):
    return len(self.img_list)

**transform and Dataloader**

In [0]:
transform = T.Compose([T.Resize(224),T.ToTensor()])
batch_size = 16
classes = os.listdir("/content/drive/My Drive/NWPU-RESISC45")
 
trainset = NWPU_RESISC45(root = "/content/drive/My Drive/train.csv",phase = 'train',transform = transform)
trainloader = DataLoader(trainset,batch_size = batch_size, shuffle = True)

validationset = NWPU_RESISC45(root = "/content/drive/My Drive/validation.csv",phase = 'validation',transform = transform)
validloader = DataLoader(validationset, batch_size = batch_size, shuffle = True)

testset = NWPU_RESISC45(root = "/content/drive/My Drive/test.csv",phase = 'test',transform = transform)
testloader = DataLoader(testset,batch_size = batch_size, shuffle = True)

**VGGNET**

In [0]:
class Net(nn.Module) :
  def __init__(self):
    super(Net,self).__init__()
    self.conv = nn.Sequential(
        #3 224 224
        nn.Conv2d(3,64,3,padding = 1),nn.ReLU(),
        nn.Conv2d(64,64,3,padding = 1),nn.ReLU(),
        nn.MaxPool2d(2,2),
        #64 112 112
        nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #128 56 56
        nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #256 28 28
        nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2),
        #512 14 14
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
        nn.MaxPool2d(2, 2)
    )
    self.linear = nn.Sequential(
        nn.Linear(512 * 7 * 7,4096),
        nn.Linear(4096,4096),
        nn.Linear(4096,45)
    )
  def forward(self,x):
    features = self.conv(x)
    x = x.view(features.size(0),-1)
    x = self.linear(x)
    return x
#cuda사용 가능하면 gpu사용, 아니면 cpu로 연산
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = Net()
net = net.cuda()

**optimizer and loss function**

In [0]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr = 0.00001)

**Train and Validation**

In [0]:
for epoch in range(100):
  running_loss = 0.0
  for i,data in enumerate(trainloader , 0):
    inputs,labels = data
    inputs = inputs.cuda()
    #labels = labels.cuda()
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
class_correct = list(0. for i in range(45))
class_total = list(0. for i in range(45))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs,_ = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1 

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))